# Section 5.1 Exponential weighted ensemble

本节我们将我们能够搜集到的全部高分结果用exponential的方法进行融合。

The forumla used is: $$exp(b*(x-S))$$

- x is the LB score of each model. Larger scores are better, hence the weights get larger as x increases and get smaller as x decreases (The better the model, the bigger the weight). 在public测试集上分数更好的预测集，我们会给他们更高的权重。


- b is the one meaningfully adjustable parameter of the model, the larger b is then the faster the weights decay as the score gets worse. 这个参数是控制这些submission的权重的，当b越大，则预测分数低的submission.csv的权重更低


- S is a calibration parameter defined such that if S is set to the best single model score then the highest unnormalised weight exp(b*(x-S)) is 1.0, which is convenient, but not essential.


- q is the sum of the unnormalised weights. Once all weights have been calculated, these weights are normalised by dividing them all by q.

## Notebooks and Results used

在这里按顺序记录我们要融合使用的结果。

- https://www.kaggle.com/code/swimmy/tuffline-amex-anotherfeaturelgbm

- https://www.kaggle.com/code/adhithyasrinivasan/amex-feature-ensemble

- https://www.kaggle.com/code/zb1373/blend-boosting-study

- https://www.kaggle.com/code/zhaoxinyu0121/amex-feature-ensemble/notebook

- https://www.kaggle.com/code/zb1373/blend-boosting-study/data?select=submission.csv

- https://www.kaggle.com/code/heyspaceturtle/beware-the-spaceturtles/notebook


**ensemble**

In [ ]:
import numpy as np
import pandas as pd
import glob

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# 修改当前文件夹位置 假定notebook文件就在项目文件夹根目录
import os
def get_root_dir():
    if os.path.exists('/content/drive/MyDrive/'):
        return '/content/drive/MyDrive/AMEX Project/notebooks' #在Colab里
    else:
        return './' #在本地

#调用系统命令，相当于cd，但是直接!cd是不行的
os.chdir(get_root_dir())

In [ ]:
b = 1000.0
S = 0.799 # 目前最高的分数
q = 0.0

导入基础模版

In [ ]:
sub = pd.read_csv("../results/0-sample-submission.csv")
display(sub)

,customer_ID,prediction
0,00000469ba478561f23a92a868bd366de6f6527a684c9a...,0
1,00001bf2e77ff879fab36aa4fac689b9ba411dae63ae39...,0
2,0000210045da4f81e5f122c6bde5c2a617d03eef67f82c...,0
3,00003b41e58ede33b8daf61ab56d9952f17c9ad1c3976c...,0
4,00004b22eaeeeb0ec976890c1d9bfc14fd9427e98c4ee9...,0
...,...,...
924616,ffff952c631f2c911b8a2a8ca56ea6e656309a83d2f64c...,0
924617,ffffcf5df59e5e0bba2a5ac4578a34e2b5aa64a1546cd3...,0
924618,ffffd61f098cc056dbd7d2a21380c4804bbfe60856f475...,0
924619,ffffddef1fc3643ea179c93245b68dca0f36941cd83977...,0


In [ ]:
scores = [0.8000,0.80000,0.79900,0.80000,0.79900,0.79900, 0.79800, 0.79800]

In [ ]:
dfs = [
    "../results/1-tuffline-plotly-amex-submission-799.csv",
    "../results/2-blend-boosting-study-submission-799.csv",
    "../results/3-amex-feature-ensemble-submission-799.csv",
    "../results/4-amex-feature-ensemble-submission-799.csv",
    "../results/5-blend-boosting-study-submission-799.csv",
    "../results/6-beware-the-spaceturtles-submission-799.csv",
    "../results/4.4-LSTM-submission.csv",
    "../results/submission_dnn_pytorch.csv"
]

In [ ]:
for df,score in zip(dfs,scores):
    df = pd.read_csv(df)
    sub["prediction"] = sub["prediction"]+df["prediction"]*np.exp(b*(score-S))
    q = q+np.exp(b*(score-S))
    
sub["prediction"] = sub["prediction"]/q
print(q)
sub.head()

11.890604367720028


,customer_ID,prediction
0,00000469ba478561f23a92a868bd366de6f6527a684c9a...,0.121086
1,00001bf2e77ff879fab36aa4fac689b9ba411dae63ae39...,0.023649
2,0000210045da4f81e5f122c6bde5c2a617d03eef67f82c...,0.123540
3,00003b41e58ede33b8daf61ab56d9952f17c9ad1c3976c...,0.289597
4,00004b22eaeeeb0ec976890c1d9bfc14fd9427e98c4ee9...,0.799599


In [ ]:
sub.to_csv("../results/8.5-ensemble5-submission.csv",index=False)